# Initialize parameters

In [1]:
using DataFrames
using CSV 
using JSON3
using JLD2
ver = "V7"
sub_n=6
file_suffix = ["OptCoords","FixedCoords","ErrbyDist"] # Three different file suffixes

df_header = ["sub","points","run",
    "avgErr_dist_0to20","stdErr_dist_0to20", "maxErr_dist_0to20","count_dist_0to20",
    "avgErr_dist_20to40","stdErr_dist_20to40", "maxErr_dist_20to40","count_dist_20to40",
    "avgErr_dist_40to60","stdErr_dist_40to60", "maxErr_dist_40to60","count_dist_40to60",
    "avgErr_dist_60plus","stdErr_dist_60plus", "maxErr_dist_60plus","count_dist_60plus"] 

19-element Vector{String}:
 "sub"
 "points"
 "run"
 "avgErr_dist_0to20"
 "stdErr_dist_0to20"
 "maxErr_dist_0to20"
 "count_dist_0to20"
 "avgErr_dist_20to40"
 "stdErr_dist_20to40"
 "maxErr_dist_20to40"
 "count_dist_20to40"
 "avgErr_dist_40to60"
 "stdErr_dist_40to60"
 "maxErr_dist_40to60"
 "count_dist_40to60"
 "avgErr_dist_60plus"
 "stdErr_dist_60plus"
 "maxErr_dist_60plus"
 "count_dist_60plus"

# Read data from file

In [2]:
# Modify this section
function comb_data_by_points(sub_n::Int64,points::Int64)
    # The total file count is set to [0, 0, 0]
    file_count = zeros(Int,3)
    # Initialize two dictionaries to store optimal coordinates and fixed coordinates, respectively
    opt_coords = Dict()
    fixed_coords = Dict()
    # Initialize data frame with appropriate column names
    comb_data = DataFrame([[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]], df_header)
    for nRuns in 1:100
        for file_type in 1:3 # Three choices [1 means OptCoords, 2 means FixedCoords, and 3 means Err]
            # Check if the file containing opt/fixed/err exists
                if(isfile("sub"*string(sub_n)*"_pt"*string(points)*"_run"*string(nRuns)*"_GridFit"*file_suffix[file_type]*".csv"))
                # Read the data from the CSV file
                data = DataFrame(CSV.File("sub"*string(sub_n)*"_pt"*string(points)*"_run"*string(nRuns)*"_GridFit"*file_suffix[file_type]*".csv"))
                if (file_type == 3)
                    append!(comb_data,data)
                else
                    # Convert data frame into a dictionary
                    grid_data = Dict(Pair.(data[!,"first"], data[!,"second"]))
                    # Sort the dictionary by key and store it in another dictionary 
                    if (file_type == 1)
                        opt_coords[file_count[file_type]+1] = sort(grid_data)
                    else
                        fixed_coords[file_count[file_type]+1] = sort(grid_data)
                    end
                end
                file_count[file_type]+=1
            end  
        end
    end
    return fixed_coords, opt_coords, comb_data
end

comb_data_by_points (generic function with 1 method)

# Store data in disk

In [3]:
function save_comb_data_by_points(sub_n::Int64,points::Int64,fixed_coords::Dict{Any,Any},opt_coords::Dict{Any,Any},errors::DataFrame)
    # Define file names
    csv_fname = "sub"*string(sub_n)*"_pt"*string(points)*"_GridFitErr_"*ver*".csv"
    json_fname = "sub"*string(sub_n)*"_pt"*string(points)*"_GridFitCoords_"*ver*".json"
    hd5_fname = "results_GridFit_"*ver*".jld2"

    fname = [csv_fname, json_fname, hd5_fname]  
    
    for f in fname
        if (occursin(".csv",f))
            # Save the data in CSV format
            CSV.write(f, errors)
        elseif (occursin(".json",f)) 
            # Save the file in JSON format
            file_type = 2
            # Merge two dictionaries (opt_coords and fixed_coords)
            coords_all = Dict("FixedCoords" => sort(fixed_coords), "OptCoords" => sort(opt_coords))
            open(json_fname, "w") do f
                JSON3.pretty(f, JSON3.write(coords_all))
            end
        else
            # save the data in HD5 format
            fixed_coords = sort(fixed_coords)
            opt_coords = sort(opt_coords)
            
            jldopen(f,"a+") do file
                file["sub"*string(sub_n)*"/pt"*string(points)*"/fixed_coords"] = sort(fixed_coords)
                file["sub"*string(sub_n)*"/pt"*string(points)*"/opt_coords"] = sort(opt_coords)
                file["sub"*string(sub_n)*"/pt"*string(points)*"/errors"] = errors
            end
        end
    end
end

save_comb_data_by_points (generic function with 1 method)

# Combine all error data by subject

In [4]:
# Run all
err_type = 3
#for sub_n in [1,2,3,5,6]
    # Initialize data frame with appropriate column names
    comb_data_by_sub = DataFrame([[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]], df_header)
    for points in [3,4,5,6,7,8,9,11,13,15]
    #points = 3
        data_by_points = comb_data_by_points(sub_n,points)
        save_comb_data_by_points(sub_n,points,data_by_points[1],data_by_points[2],data_by_points[3])
        
        # Append all data here and save in JSON/CSV format
        append!(comb_data_by_sub,data_by_points[err_type])
    end
    # Write combined error data to a CSV file
    if !(isempty(comb_data_by_sub))
        CSV.write("sub"*string(sub_n)*"_GridFitErr_"*ver*".csv", comb_data_by_sub)     
    end
#end

"sub6_GridFitErr_V7_ext.csv"

# Verify the HD5 file that contains all data in one place

In [5]:
hd5_fname = "results_GridFit_"*ver*".jld2"
file = jldopen(hd5_fname, "r")

JLDFile /mnt/data/msaber/git/data/GridFit_dataV7_ext/results_sub6/results_GridFit_V7_ext.jld2 (read-only)
 ├─📂 sub1
 │  ├─📂 pt3
 │  │  ├─🔢 fixed_coords
 │  │  ├─🔢 opt_coords
 │  │  └─🔢 errors
 │  ├─📂 pt4
 │  │  ├─🔢 fixed_coords
 │  │  └─ ⋯ (2 more entries)
 │  └─ ⋯ (8 more entries)
 └─ ⋯ (4 more entries)